Introduction to Bioinformatics

HomeWork 3

Statistical tests

name: "Mehrshad Barzamini"

student id: "99170361"



In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to the folder containing the dataset
import os
dataset_path = '/content/drive/MyDrive/Education/Bioinformatics/Introduction_To_Bioinformatics_Fall_2024/HWs/HW3/Practical/GSE10072_series_matrix.txt.gz'

# Unzip the file
import gzip
import shutil

# Define output file path
output_file = '/content/GSE10072_series_matrix.txt'

# Unzip the .gz file
with gzip.open(dataset_path, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("File extracted successfully!")


Mounted at /content/drive
File extracted successfully!


# Gene Expression Analysis Using Statistical Tests

This notebook guides you through analyzing a gene expression dataset to identify significant differences between two groups: **Lung Tumor** and **Normal Lung**. It includes the following key steps:

- **Loading the dataset**: Familiarize yourself with the data structure.
- **Descriptive statistics**: Calculate basic measures such as means and standard deviations.
- **Hypothesis testing**: Use statistical tests (e.g., t-tests) to identify differences between the groups.
- **Multiple testing correction**: Adjust p-values to control the False Discovery Rate.
- **Filtering significant genes**: Identify and prioritize statistically significant genes.

Some sections include **###TO DO tasks**, where you'll complete the code or analysis to achieve specific results.


## Step 1: Load the Dataset

The dataset contains gene expression levels for samples classified as **Lung Tumor** and **Normal Lung**. Each row corresponds to a specific gene, and columns represent expression levels across samples.

In this step, we load the dataset and display the first few rows to understand its structure.

In [2]:
## Step 1: Load the Dataset

# Import necessary libraries
import pandas as pd

# Define the path to the extracted dataset
file_path = '/content/GSE10072_series_matrix.txt'

# Load the dataset, skipping initial comment lines starting with '!'
data = pd.read_csv(file_path, sep='\t', comment='!', index_col=0)

# Display the first 5 rows of the dataset
print("Dataset Overview:")
print(data.shape)  # Print dataset dimensions
print(data.head())  # Display first 5 rows


Dataset Overview:
(22283, 107)
           GSM254625  GSM254626  GSM254627  GSM254628  GSM254629  GSM254630  \
ID_REF                                                                        
1007_s_at  10.927084  10.416978  10.628538  10.151180  10.988512  10.778205   
1053_at     6.895217   6.924856   7.550245   6.699557   6.826031   6.718372   
117_at      8.110190   7.760228   7.974676   7.712676   7.775592   7.777087   
121_at      9.451286   9.520943   9.807597   9.522087   9.855061   9.861055   
1255_g_at   4.814477   4.718640   4.905163   4.818076   4.823958   4.848313   

           GSM254631  GSM254632  GSM254633  GSM254634  ...  GSM254722  \
ID_REF                                                 ...              
1007_s_at  10.568814  10.479425  10.548843  10.465005  ...  10.727493   
1053_at     6.739765   6.724615   7.102475   6.627922  ...   6.856575   
117_at      7.893210   8.058398   8.005438   7.961476  ...   7.741922   
121_at     10.126183   9.878970  10.110318  10.411

## Step 2: Assign Sample Groups

The samples are divided into two groups:
- **Normal Lung**: Includes expression levels for normal lung tissue.
- **Lung Tumor**: Includes expression levels for lung tumor tissue.

The corresponding sample names are assigned to these groups based on metadata from the dataset.


In [3]:
## DO NOT Change this cell

normal_lung = [
    "GSM254626", "GSM254628", "GSM254632", "GSM254634", "GSM254635",
    "GSM254638", "GSM254640", "GSM254643", "GSM254644", "GSM254646",
    "GSM254649", "GSM254651", "GSM254653", "GSM254655", "GSM254658",
    "GSM254660", "GSM254662", "GSM254665", "GSM254667", "GSM254669",
    "GSM254671", "GSM254673", "GSM254676", "GSM254677", "GSM254679",
    "GSM254681", "GSM254683", "GSM254685", "GSM254689", "GSM254691",
    "GSM254693", "GSM254695", "GSM254699", "GSM254702", "GSM254703",
    "GSM254706", "GSM254708", "GSM254710", "GSM254711", "GSM254712",
    "GSM254713", "GSM254715", "GSM254717", "GSM254719", "GSM254723",
    "GSM254725", "GSM254727", "GSM254730", "GSM254731"
]

lung_tumor = [
    "GSM254625", "GSM254627", "GSM254629", "GSM254630", "GSM254631",
    "GSM254633", "GSM254636", "GSM254637", "GSM254639", "GSM254641",
    "GSM254642", "GSM254645", "GSM254647", "GSM254648", "GSM254650",
    "GSM254652", "GSM254654", "GSM254656", "GSM254657", "GSM254659",
    "GSM254661", "GSM254663", "GSM254664", "GSM254666", "GSM254668",
    "GSM254670", "GSM254672", "GSM254674", "GSM254675", "GSM254678",
    "GSM254680", "GSM254682", "GSM254684", "GSM254686", "GSM254687",
    "GSM254688", "GSM254690", "GSM254692", "GSM254694", "GSM254696",
    "GSM254697", "GSM254698", "GSM254700", "GSM254701", "GSM254704",
    "GSM254705", "GSM254707", "GSM254709", "GSM254714", "GSM254716",
    "GSM254718", "GSM254720", "GSM254721", "GSM254722", "GSM254724",
    "GSM254726", "GSM254728", "GSM254729"
]

## Step 3: Calculate Descriptive Statistics

For each gene, we calculate:
- **Mean** expression levels for the two groups.
- **Standard Deviation (Std)** to understand the variability within each group.

These statistics provide an initial overview of the data distribution.


In [4]:
# Calculate means for each group
mean_normal = data[normal_lung].mean(axis=1)
mean_tumor = data[lung_tumor].mean(axis=1)

# Combine into a DataFrame
mean_df = pd.DataFrame({
    'Mean_Normal_Lung': mean_normal,
    'Mean_Lung_Tumor': mean_tumor
})

# Display the updated DataFrame (mean columns)
print(mean_df.shape)
print(mean_df.head())

(22283, 2)
           Mean_Normal_Lung  Mean_Lung_Tumor
ID_REF                                      
1007_s_at         10.309772        10.772500
1053_at            6.736384         6.988408
117_at             8.007275         8.016987
121_at             9.862801         9.878110
1255_g_at          4.857075         4.923281


In [5]:
# Calculate standard deviations for each group
std_normal = data[normal_lung].std(axis=1)
std_tumor = data[lung_tumor].std(axis=1)

# Combine into a DataFrame
std_df = pd.DataFrame({
    'Std_Normal_Lung': std_normal,
    'Std_Lung_Tumor': std_tumor
})

# Display the updated DataFrame (std columns)
print(std_df.shape)
print(std_df.head())

(22283, 2)
           Std_Normal_Lung  Std_Lung_Tumor
ID_REF                                    
1007_s_at         0.231752        0.364252
1053_at           0.137845        0.270940
117_at            0.463988        0.275375
121_at            0.171649        0.170197
1255_g_at         0.125765        0.138166


In [6]:
# Merge the mean and std DataFrames
result_df = pd.concat([mean_df, std_df], axis=1)

# Display the updated DataFrame (mean columns and std columns)
print(result_df.shape)
print(result_df.head())

(22283, 4)
           Mean_Normal_Lung  Mean_Lung_Tumor  Std_Normal_Lung  Std_Lung_Tumor
ID_REF                                                                       
1007_s_at         10.309772        10.772500         0.231752        0.364252
1053_at            6.736384         6.988408         0.137845        0.270940
117_at             8.007275         8.016987         0.463988        0.275375
121_at             9.862801         9.878110         0.171649        0.170197
1255_g_at          4.857075         4.923281         0.125765        0.138166


## Step 4: Calculate Log2 Fold Change (Log2FC)

Log2 Fold Change $( \log_2 \text{FC} )$ measures the relative difference in expression levels between the **Lung Tumor** and **Normal Lung** groups for each gene. It is calculated as:
$$
\text{Log2FC} = \log_2\left(\frac{\text{Mean Lung Tumor}}{\text{Mean Normal Lung}}\right)
$$

Genes with a large positive or negative Log2FC indicate stronger differences in expression levels between the two groups.


In [24]:
import numpy as np

# Ensure the columns are in the correct order
mean_df = mean_df[['Mean_Lung_Tumor', 'Mean_Normal_Lung']]  # Explicitly reorder

# Compute Log2 Fold Change (Log2FC) for each gene
log2fc = np.log2(mean_df['Mean_Lung_Tumor'] / mean_df['Mean_Normal_Lung'])

# Add Log2FC to the DataFrame
log2fc_df = mean_df.copy()
log2fc_df['Log2FC'] = log2fc

# Display the updated DataFrame with Log2FC (mean columns and Log2FC columns)
print(log2fc_df.shape)
print(log2fc_df.head())

(22283, 3)
           Mean_Lung_Tumor  Mean_Normal_Lung    Log2FC
ID_REF                                                
1007_s_at        10.772500         10.309772  0.063341
1053_at           6.988408          6.736384  0.052990
117_at            8.016987          8.007275  0.001749
121_at            9.878110          9.862801  0.002238
1255_g_at         4.923281          4.857075  0.019533


In [25]:
### Apply new threshold for Log2FC
threshold = 0.25  # New threshold value

# Filter genes based on the new threshold
filtered_df = log2fc_df[(log2fc_df['Log2FC'] > threshold) | (log2fc_df['Log2FC'] < -threshold)]

# Display the number of filtered genes
print(f"Number of genes with |Log2FC| > {threshold}: {filtered_df.shape[0]}")

# Display the filtered genes
print(filtered_df.head())

Number of genes with |Log2FC| > 0.25: 349
             Mean_Lung_Tumor  Mean_Normal_Lung    Log2FC
ID_REF                                                  
200878_at          10.997457         13.084882 -0.250731
201013_s_at        10.029449          8.360345  0.262608
201014_s_at         8.654871          6.880522  0.330994
201291_s_at         7.792620          5.370544  0.537040
201292_at           8.510264          6.054301  0.491243


In [26]:
# Sort the filtered genes by Log2FC in descending order
sorted_df = filtered_df.sort_values(by='Log2FC', ascending=False)

# Display the sorted genes
print(sorted_df.shape)  # Display the dimensions of the sorted DataFrame
print(sorted_df.head(10))  # Display the top 10 genes with the highest Log2FC
print(sorted_df.tail(10))  # Display the bottom 10 genes with the lowest Log2FC

(349, 3)
             Mean_Lung_Tumor  Mean_Normal_Lung    Log2FC
ID_REF                                                  
209875_s_at        11.697897          7.333482  0.673679
204475_at           7.852579          4.990543  0.653970
37892_at            8.718949          5.657426  0.624008
214774_x_at         7.281510          4.843550  0.588173
218468_s_at         7.513001          5.071543  0.566964
201291_s_at         7.792620          5.370544  0.537040
218469_at           8.418868          5.870505  0.520142
217428_s_at         8.325766          5.862291  0.506118
206239_s_at         9.403921          6.645521  0.500880
216623_x_at         7.961952          5.650636  0.494709
             Mean_Lung_Tumor  Mean_Normal_Lung    Log2FC
ID_REF                                                  
217046_s_at         6.467130          9.504819 -0.555534
209904_at           5.668670          8.566648 -0.595720
219230_at           6.868648         10.438961 -0.603880
210096_at           7.

## Step 5: Perform Hypothesis Testing

We perform an independent two-sample t-test for each gene to compare expression levels between the two groups:
- Null Hypothesis (\(H_0\)): There is no significant difference in mean expression levels between the groups.
- Alternative Hypothesis (\(H_a\)): There is a significant difference in mean expression levels between the groups.

The p-values resulting from these tests indicate the statistical significance of the differences.


In [27]:
from scipy.stats import ttest_ind

# Initialize an empty list to store p-values
p_values = []

# Perform t-test for each gene
for gene in data.index:
    # Extract expression levels for the two groups
    group1 = data.loc[gene, normal_lung]  # Normal Lung group
    group2 = data.loc[gene, lung_tumor]  # Lung Tumor group

    # Perform independent two-sample t-test
    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)  # Welch's t-test
    p_values.append(p_val)

# Add p-values as a new column in the DataFrame
log2fc_df['P_Value'] = p_values

# Display the updated DataFrame with P-Value (mean columns and P-value column)
print(log2fc_df.shape)
print(log2fc_df.head())


(22283, 4)
           Mean_Lung_Tumor  Mean_Normal_Lung    Log2FC       P_Value
ID_REF                                                              
1007_s_at        10.772500         10.309772  0.063341  3.167943e-12
1053_at           6.988408          6.736384  0.052990  1.825086e-08
117_at            8.016987          8.007275  0.001749  8.979953e-01
121_at            9.878110          9.862801  0.002238  6.454791e-01
1255_g_at         4.923281          4.857075  0.019533  1.087906e-02


## Step 6: Adjust P-Values for Multiple Testing

To account for multiple comparisons, we apply the Benjamini-Hochberg procedure to control the false discovery rate (FDR). This ensures that the reported significant genes are less likely to include false positives.

Adjusted p-values (\(q\)-values) are calculated for each gene.


In [28]:
from statsmodels.stats.multitest import multipletests

# Adjust p-values using Benjamini-Hochberg correction
p_adjusted = multipletests(log2fc_df['P_Value'], method='fdr_bh')[1]

# Add adjusted p-values as a new column in the DataFrame
log2fc_df['Adj_P_Value'] = p_adjusted

# Display the updated DataFrame with adjusted p-values
print(log2fc_df.shape)
print(log2fc_df.head())


(22283, 5)
           Mean_Lung_Tumor  Mean_Normal_Lung    Log2FC       P_Value  \
ID_REF                                                                 
1007_s_at        10.772500         10.309772  0.063341  3.167943e-12   
1053_at           6.988408          6.736384  0.052990  1.825086e-08   
117_at            8.016987          8.007275  0.001749  8.979953e-01   
121_at            9.878110          9.862801  0.002238  6.454791e-01   
1255_g_at         4.923281          4.857075  0.019533  1.087906e-02   

            Adj_P_Value  
ID_REF                   
1007_s_at  4.047664e-11  
1053_at    1.199304e-07  
117_at     9.325485e-01  
121_at     7.426276e-01  
1255_g_at  2.414763e-02  


## Step 7: Filter and Sort Significant Genes

Genes with adjusted p-values (\(q\)-values) below a significance threshold (e.g., 0.05) are considered statistically significant.
These genes are sorted in ascending order of adjusted p-values to prioritize the most significant results.


In [30]:
# Define thresholds
adj_pval_threshold = 0.05  # Adjusted P-Value threshold
log2fc_threshold = 0.25     # Log2FC threshold

# Filter significant genes based on criteria
significant_genes = log2fc_df[
    (log2fc_df['Adj_P_Value'] < adj_pval_threshold) &  # Adjusted P-value < 0.05
    (abs(log2fc_df['Log2FC']) > log2fc_threshold)     # |Log2FC| > 1
]

# Sort significant genes by adjusted p-value in ascending order
sorted_significant_genes = significant_genes.sort_values(by='Adj_P_Value', ascending=True)

# Display the top 10 significant genes
print(f"Number of significant genes: {sorted_significant_genes.shape[0]}")
print(sorted_significant_genes.head(10))  # Display top 10 genes


Number of significant genes: 349
             Mean_Lung_Tumor  Mean_Normal_Lung    Log2FC       P_Value  \
ID_REF                                                                   
209074_s_at         7.451084         10.822883 -0.538563  6.641899e-44   
204396_s_at         7.583586          9.971658 -0.394953  2.009124e-41   
204677_at           6.804586          9.424160 -0.469857  1.317136e-40   
210081_at           7.459343         11.876812 -0.671027  4.769095e-38   
202112_at           9.567466         11.825939 -0.305746  1.464679e-37   
209875_s_at        11.697897          7.333482  0.673679  1.782532e-37   
203980_at           7.136529         10.975070 -0.620936  2.254575e-37   
204719_at           6.484080          9.443989 -0.542495  2.925072e-37   
205200_at           7.338750         10.750604 -0.550811  3.251662e-37   
209555_s_at         7.154256          9.571239 -0.419904  3.311714e-37   

              Adj_P_Value  
ID_REF                     
209074_s_at  1.480014e